In [ ]:
!pip install lightautoml

     |████████████████████████████████| 293 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 47.5 MB/s 
     |████████████████████████████████| 204 kB 65.5 MB/s 
     |████████████████████████████████| 804.1 MB 1.8 kB/s 
     |████████████████████████████████| 308 kB 45.8 MB/s 
     |████████████████████████████████| 76.3 MB 22 kB/s 
     |████████████████████████████████| 424 kB 46.5 MB/s 
     |████████████████████████████████| 15.7 MB 55 kB/s 
     |████████████████████████████████| 2.8 MB 35.1 MB/s 
     |████████████████████████████████| 80 kB 6.9 MB/s 
     |████████████████████████████████| 209 kB 55.3 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 148 kB 61.3 MB/s 
     |████████████████████████████████| 112 kB 53.8 MB/s 
     |████████████████████████████████| 575 kB 43.5 MB/s 
     |████████████████████████████████| 23.3 MB 81 kB/s 
     |█████████████████

In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
import pandas as pd
# Standard python libraries
import logging
import os
import time
import requests
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)
import lightautoml

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

import multiprocessing as mp

# set option of max 100 displayed rows
pd.set_option('display.max_rows', 200)

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
konh = pd.read_csv('/content/konh.csv')

In [ ]:
konh.describe()

,Unnamed: 0,id_client,id_contract,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,...,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET
count,8922.000000,8922.000000,8922.000000,8922.000000,3.813000e+03,5.390000e+02,3.618000e+03,1.240000e+02,7.180000e+02,1.029000e+03,6.870000e+02,6.238000e+03,5.255000e+03,1.483000e+03,6.002000e+03,1.243000e+03,6.101000e+03,1.975000e+03,6.240000e+03,3.778000e+03,3.600000e+01,5.560000e+02,6.410000e+02,3.749000e+03,2.127000e+03,1.312000e+03,9.900000e+02,3.780000e+02,6.083000e+03,2.587000e+03,6.016000e+03,4.560000e+02,4.760000e+02,6.255000e+03,6.254000e+03,3.804000e+03,6.104000e+03,5.933000e+03,6.117000e+03,1.842000e+03,...,828.000000,1315.000000,1492.000000,2253.000000,2204.000000,1.823000e+03,2791.000000,2737.000000,2.336000e+03,3608.000000,3599.000000,3.402000e+03,3753.000000,3749.000000,3.589000e+03,2253.000000,1255.000000,2075.000000,1974.000000,1972.000000,3611.000000,2218.000000,3533.000000,3464.000000,3461.000000,3.144000e+03,5.230000e+03,8.220000e+02,1.770000e+03,5.231000e+03,5.287000e+03,5.116000e+03,5.303000e+03,1.536000e+03,1.208000e+03,5.324000e+03,1.030000e+03,1.241000e+03,5.328000e+03,8922.000000
mean,4460.500000,6367.712845,8927.420745,0.183577,1.500905e+09,6.935237e+07,1.060895e+09,6.227641e+07,1.455451e+09,8.564511e+07,7.738869e+08,4.514681e+08,8.781025e+07,1.437572e+07,2.746508e+08,2.235322e+08,5.912421e+07,2.321999e+07,6.457392e+08,4.102052e+08,-4.440332e+07,1.318131e+09,5.973565e+07,2.541220e+08,8.580067e+08,1.110346e+09,1.869001e+08,3.580622e+08,4.414704e+08,2.593900e+08,3.033594e+08,1.524438e+08,7.371261e+07,1.364650e+09,1.365276e+09,4.936527e+08,1.347639e+09,1.068106e+09,8.740586e+07,6.287075e+08,...,inf,inf,inf,inf,0.628502,2.003402e+06,inf,0.625971,1.945206e+06,inf,0.612714,1.809537e+06,inf,0.606056,1.759142e+06,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,5.279166e+07,1.709445e+07,9.222022e+07,3.161784e+07,1.690577e+07,1.343356e+08,1.112911e+08,5.207869e+06,5.993611e+07,1.559127e+07,2.901361e+06,-8.492533e+05,-4.948053e+06,-3.212004e+05,0.209786
std,2575.703884,3680.554333,4881.111898,0.385001,2.494452e+10,6.422873e+08,1.698762e+10,2.631264e+08,1.343161e+10,5.648259e+08,1.127785e+10,3.965542e+09,9.081734e+08,1.297399e+08,2.495538e+09,1.856772e+09,7.509933e+08,4.124585e+08,1.472846e+10,1.111992e+10,1.491933e+08,7.990794e+09,7.846531e+08,4.443357e+09,1.238759e+10,1.339969e+10,2.008815e+09,1.761483e+09,4.088649e+09,2.657827e+09,2.543953e+09,5.972178e+08,8.306361e+08,2.283372e+10,2.283563e+10,5.705677e+09,1.087690e+10,7.715642e+09,1.851821e+09,6.120271e+09,...,NaN,NaN,NaN,NaN,0.350524,1.290116e+07,NaN,0.326155,8.084966e+06,NaN,0.300722,1.153111e+07,NaN,0.275773,1.360440e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.286358e+09,3.551110e+08,1.854702e+09,1.015829e+09,4.007758e+08,1.144258e+09,8.639862e+08,6.034318e+08,8.888367e+08,1.075308e+08,1.206665e+09,1.053921e+08,1.904253e+08,1.133635e+09,0.402818
min,0.000000,1.000000,2.000000,0.000000,-1.820000e+05,1.000000e+03,1.000000e+03,-1.175000e+06,1.000000e+03,-2.540000e+05,1.000000e+03,-2.950000e+05,-2.100000e+05,-3.990000e+05,1.000000e+03,-5.200000e+04,-2.103100e+07,-3.701000e+06,-7.865320e+10,1.000000e+03,-8.538600e+08,-2.273630e+08,1.000000e+03,-7.865321e+10,-4.140000e+06,-4.140000e+06,-7.150000e+05,-2.427000e+06,-3.519000e+06,-1.998900e+07,-1.022000e+06,-5.240000e+05,-8.498100e+07,-2.950000e+05,-2.950000e+05,-6.795270e+09,3.000000e+03,-1.110870e+08,-6.858694e+09,1.000000e+03,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.0000

In [ ]:
l_d = pd.read_csv('/content/last_df_mkd(5.11).csv')

In [ ]:
l_d = l_d.drop(['Unnamed: 0'],axis=1)

In [ ]:
konh = konh.drop(['Unnamed: 0'],axis=1)

In [ ]:
obj_df = l_d.loc[:,['id_client','IP_flag','EGRPOINCLUDED','OKFS_GROUP','OKOPF_GROUP','OKOGU_GROUP','OKATO_REGIONCODE','OKATO_FED','OKTMO_CODE_fixed','OKTMO_FED','WORKERSRANGE','OKVED_CODE_fixed','FLAG_DISQUALIFICATION','SEX_NAME','CITIZENSHIP_NAME','TELEPHONECOUNT_fixed','MANAGERCOUNTINCOUNTRY_fixed','MANAGERCOUNTINREGION_fixed','MANAGERINNCOUNT_fixed']]

In [ ]:
obj_df = obj_df.astype('category')

In [ ]:
id = konh.id_client.array

In [ ]:
new_obj_df = pd.DataFrame(columns=obj_df.columns)

In [ ]:
for x in id:
    new_obj_df = new_obj_df.append(obj_df[obj_df.id_client==x][0:1])


In [ ]:
obj_df.sort_values(by='id_client')

,id_client,IP_flag,EGRPOINCLUDED,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE,OKATO_FED,OKTMO_CODE_fixed,OKTMO_FED,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
5165,1,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
2388,2,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
17328,3,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
19083,4,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,51 .. 100,18.12,NaN,NaN,NaN,1.0,3.0,1.0,1.0
15085,5,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10257,12731,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,45.31.1,NaN,NaN,NaN,3.0,3.0,3.0,3.0
10004,12732,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,68.20,NaN,NaN,NaN,3.0,3.0,3.0,3.0
5290,12733,0,1.0,Иностранная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,11 .. 15,46.71,NaN,NaN,NaN,2.0,2.0,2.0,2.0
3303,12734,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,55.0,NaN,RARE_VALUE,NaN,0 .. 5,RARE_VALUE,NaN,NaN,NaN,1.0,2.0,1.0,2.0


In [ ]:
konh  = konh.drop(['OKTMO_CODE'],axis=1)

In [ ]:
last_train = pd.merge(konh, new_obj_df, on='id_client', how='outer')

In [ ]:
last_train = last_train.drop(['IP_flag_x'],axis=1)

In [ ]:
last_train

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
0,1,5166.0,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,6121000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,924000.0,NaN,924000.0,NaN,NaN,7045000.0,7045000.0,NaN,32078000.0,27921000.0,4157000.0,NaN,NaN,4157000.0,...,NaN,NaN,6.050000,2.00,5.045455,9.000000,4.686366,0.0,NaN,NaN,NaN,38000.0,5472000.0,4554000.0,918000.0,NaN,NaN,918000.0,NaN,NaN,918000.0,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,2,2389.0,14626000.0,NaN,14414000.0,NaN,212000.0,NaN,NaN,14939000.0,8521000.0,NaN,6395000.0,NaN,23000.0,NaN,18901000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10664000.0,8995000.0,1669000.0,NaN,NaN,29565000.0,29565000.0,NaN,28874000.0,26330000.0,2544000.0,NaN,NaN,2126000.0,...,NaN,NaN,4.190476,2.00,3.153846,3.153846,4.306454,-1180000.0,6213000.0,NaN,8525000.0,19000.0,17403000.0,16235000.0,1168000.0,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
2,3,17329.0,3745000.0,NaN,3745000.0,NaN,NaN,NaN,NaN,9131000.0,888000.0,67000.0,6261000.0,NaN,645000.0,1270000.0,40000.0,40000.0,NaN,NaN,NaN,NaN,10113000.0,10113000.0,NaN,NaN,2723000.0,NaN,2723000.0,NaN,NaN,12876000.0,12876000.0,120000.0,378000.0,258000.0,73000.0,2000.0,45000.0,-1000.0,...,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,1005000.0,-2225000.0,-2391000.0,NaN,2440000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
3,5,15086.0,87239000.0,NaN,86087000.0,NaN,NaN,262000.0,890000.0,75181000.0,23412000.0,NaN,29428000.0,10200000.0,12141000.0,NaN,134209000.0,10518000.0,NaN,NaN,NaN,104475000.0,NaN,NaN,NaN,NaN,28211000.0,NaN,28111000.0,NaN,100000.0,162420000.0,162420000.0,104340000.0,871384000.0,767044000.0,40236000.0,64104000.0,NaN,26446000.0,...,17.000000,11.737408,6.058442,3.00,5.937716,5.023102,6.189865,9112000.0,2017000.0,NaN,NaN,-1121000.0,23631000.0,-7613000.0,24452000.0,6792000.0,NaN,23302000.0,256000.0,NaN,19022000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
last_train = last_train.drop(['FLAG_DISQUALIFICATION_x','EGRPOINCLUDED_x','OKATO_REGIONCODE_x','OKATO_FED_x','OKTMO_FED_x'],axis=1)

In [ ]:
sorted(list(last_train.new4.unique()))[-2]
l = [4,5,6,7,10,13,16,19,20,21,22,23,24,25,26,27,28]
def min_inf(x):
    if x==np.inf:
        return m+100
    return x
for x in l:
    m = sorted(list(last_train['new'+str(x)].unique()))[-2]
    last_train['new'+str(x)] = last_train['new'+str(x)].apply(min_inf)


In [ ]:
last_train['new3'].describe()

count    1.853000e+03
mean     9.178187e+05
std      5.079952e+06
min      0.000000e+00
25%      2.086000e+03
50%      6.148100e+04
75%      3.453702e+05
max      1.122622e+08
Name: new3, dtype: float64

In [ ]:
l_d['new0'].describe()

count    3.615000e+03
mean     1.730700e+06
std      7.173170e+06
min      0.000000e+00
25%      9.400375e+04
50%      1.368927e+05
75%      5.496305e+05
max      1.181978e+08
Name: new0, dtype: float64

In [ ]:
last_train

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed
0,1,5166.0,10000.0,NaN,10000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,NaN,NaN,NaN,7035000.0,NaN,6121000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,924000.0,NaN,924000.0,NaN,NaN,7045000.0,7045000.0,NaN,32078000.0,27921000.0,4157000.0,NaN,NaN,4157000.0,...,NaN,NaN,6.050000,2.00,5.045455,9.000000,4.686366,0.0,NaN,NaN,NaN,38000.0,5472000.0,4554000.0,918000.0,NaN,NaN,918000.0,NaN,NaN,918000.0,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,2,2389.0,14626000.0,NaN,14414000.0,NaN,212000.0,NaN,NaN,14939000.0,8521000.0,NaN,6395000.0,NaN,23000.0,NaN,18901000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10664000.0,8995000.0,1669000.0,NaN,NaN,29565000.0,29565000.0,NaN,28874000.0,26330000.0,2544000.0,NaN,NaN,2126000.0,...,NaN,NaN,4.190476,2.00,3.153846,3.153846,4.306454,-1180000.0,6213000.0,NaN,8525000.0,19000.0,17403000.0,16235000.0,1168000.0,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0
2,3,17329.0,3745000.0,NaN,3745000.0,NaN,NaN,NaN,NaN,9131000.0,888000.0,67000.0,6261000.0,NaN,645000.0,1270000.0,40000.0,40000.0,NaN,NaN,NaN,NaN,10113000.0,10113000.0,NaN,NaN,2723000.0,NaN,2723000.0,NaN,NaN,12876000.0,12876000.0,120000.0,378000.0,258000.0,73000.0,2000.0,45000.0,-1000.0,...,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,1005000.0,-2225000.0,-2391000.0,NaN,2440000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0
3,5,15086.0,87239000.0,NaN,86087000.0,NaN,NaN,262000.0,890000.0,75181000.0,23412000.0,NaN,29428000.0,10200000.0,12141000.0,NaN,134209000.0,10518000.0,NaN,NaN,NaN,104475000.0,NaN,NaN,NaN,NaN,28211000.0,NaN,28111000.0,NaN,100000.0,162420000.0,162420000.0,104340000.0,871384000.0,767044000.0,40236000.0,64104000.0,NaN,26446000.0,...,17.000000,11.737408,6.058442,3.00,5.937716,5.023102,6.189865,9112000.0,2017000.0,NaN,NaN,-1121000.0,23631000.0,-7613000.0,24452000.0,6792000.0,NaN,23302000.0,256000.0,NaN,19022000.0,0.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,12731,10258.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
new = 'new_F1150	new_F1230	new_F1410	new_F1510	new_F1520	new_F2110	new_F2120	new_F2200	new_F2210	new_F2220	new_F2300	new_F2320	new_F2330'
new = new.split()
new = new + ['new0','new1','new2','new3','new9','new12','new15','new18']

In [ ]:
for x in new:
    m = sorted(list(last_train[x].unique()))[-2]
    last_train[x] = last_train[x].apply(min_inf)

In [ ]:
last_train.describe()

,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,F2310,...,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET
count,8922.000000,3.813000e+03,5.390000e+02,3.618000e+03,1.240000e+02,7.180000e+02,1.029000e+03,6.870000e+02,6.238000e+03,5.255000e+03,1.483000e+03,6.002000e+03,1.243000e+03,6.101000e+03,1.975000e+03,6.240000e+03,3.778000e+03,3.600000e+01,5.560000e+02,6.410000e+02,3.749000e+03,2.127000e+03,1.312000e+03,9.900000e+02,3.780000e+02,6.083000e+03,2.587000e+03,6.016000e+03,4.560000e+02,4.760000e+02,6.255000e+03,6.254000e+03,3.804000e+03,6.104000e+03,5.933000e+03,6.117000e+03,1.842000e+03,1.584000e+03,6.128000e+03,1.060000e+02,...,828.000000,1315.000000,1492.000000,2253.000000,2204.000000,1.823000e+03,2791.000000,2737.000000,2.336000e+03,3608.000000,3599.000000,3.402000e+03,3753.000000,3749.000000,3.589000e+03,2253.000000,1255.000000,2075.000000,1974.000000,1972.000000,3611.000000,2218.000000,3533.000000,3464.000000,3461.000000,3.144000e+03,5.230000e+03,8.220000e+02,1.770000e+03,5.231000e+03,5.287000e+03,5.116000e+03,5.303000e+03,1.536000e+03,1.208000e+03,5.324000e+03,1.030000e+03,1.241000e+03,5.328000e+03,8922.000000
mean,8927.420745,1.500905e+09,6.935237e+07,1.060895e+09,6.227641e+07,1.455451e+09,8.564511e+07,7.738869e+08,4.514681e+08,8.781025e+07,1.437572e+07,2.746508e+08,2.235322e+08,5.912421e+07,2.321999e+07,6.457392e+08,4.102052e+08,-4.440332e+07,1.318131e+09,5.973565e+07,2.541220e+08,8.580067e+08,1.110346e+09,1.869001e+08,3.580622e+08,4.414704e+08,2.593900e+08,3.033594e+08,1.524438e+08,7.371261e+07,1.364650e+09,1.365276e+09,4.936527e+08,1.347639e+09,1.068106e+09,8.740586e+07,6.287075e+08,1.259131e+08,8.280170e+07,4.010269e+08,...,66.023012,52.398556,33.130693,2.567441,0.628502,2.003402e+06,2.197177,0.625971,1.945206e+06,0.325049,0.612714,1.809537e+06,0.174565,0.606056,1.759142e+06,7.114980,57.970627,15.976995,16.228886,177.834909,7.447513,68.884507,22.333758,16.989016,87.629862,5.279166e+07,1.709445e+07,9.222022e+07,3.161784e+07,1.690577e+07,1.343356e+08,1.112911e+08,5.207869e+06,5.993611e+07,1.559127e+07,2.901361e+06,-8.492533e+05,-4.948053e+06,-3.212004e+05,0.209786
std,4881.111898,2.494452e+10,6.422873e+08,1.698762e+10,2.631264e+08,1.343161e+10,5.648259e+08,1.127785e+10,3.965542e+09,9.081734e+08,1.297399e+08,2.495538e+09,1.856772e+09,7.509933e+08,4.124585e+08,1.472846e+10,1.111992e+10,1.491933e+08,7.990794e+09,7.846531e+08,4.443357e+09,1.238759e+10,1.339969e+10,2.008815e+09,1.761483e+09,4.088649e+09,2.657827e+09,2.543953e+09,5.972178e+08,8.306361e+08,2.283372e+10,2.283563e+10,5.705677e+09,1.087690e+10,7.715642e+09,1.851821e+09,6.120271e+09,1.090643e+09,1.943175e+09,2.906663e+09,...,71.635720,94.222356,69.137279,15.752301,0.350524,1.290116e+07,14.540538,0.326155,8.084966e+06,5.491525,0.300722,1.153111e+07,3.731052,0.275773,1.360440e+07,23.769449,85.473594,46.329588,43.767608,567.264443,43.580584,105.868447,135.570288,88.507205,592.250837,1.286358e+09,3.551110e+08,1.854702e+09,1.015829e+09,4.007758e+08,1.144258e+09,8.639862e+08,6.034318e+08,8.888367e+08,1.075308e+08,1.206665e+09,1.053921e+08,1.904253e+08,1.133635e+09,0.402818
min,2.000000,-1.820000e+05,1.000000e+03,1.000000e+03,-1.175000e+06,1.000000e+03,-2.540000e+05,1.000000e+03,-2.950000e+05,-2.100000e+05,-3.990000e+05,1.000000e+03,-5.200000e+04,-2.103100e+07,-3.701000e+06,-7.865320e+10,1.000000e+03,-8.538600e+08,-2.273630e+08,1.000000e+03,-7.865321e+10,-4.140000e+06,-4.140000e+06,-7.150000e+05,-2.427000e+06,-3.519000e+06,-1.998900e+07,-1.022000e+06,-5.240000e+05,-8.498100e+07,-2.950000e+05,-2.950000e+05,-6.795270e+09,3.000000e+03,-1.110870e+

In [ ]:
last_train.TARGET.value_counts()

0.000000    6977
1.000000    1791
0.500000      64
0.333333      20
0.666667      19
0.750000       9
0.250000       7
0.875000       4
0.800000       3
0.166667       3
0.200000       3
0.900000       2
0.888889       2
0.125000       1
0.026667       1
0.090909       1
0.927273       1
0.939394       1
0.285714       1
0.909091       1
0.100000       1
0.933333       1
0.937500       1
0.969697       1
0.700000       1
0.833333       1
0.058824       1
0.003413       1
0.600000       1
0.941176       1
0.916667       1
Name: TARGET, dtype: int64

In [ ]:
konh.TARGET

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
8917    1.0
8918    0.0
8919    0.0
8920    1.0
8921    0.0
Name: TARGET, Length: 8922, dtype: float64

In [ ]:
import scipy.stats as st

In [ ]:
le_ri = []
def per_fil(x):
    if x<l:
        return l
    if x>r:
        return r
    return x
for x in last_train.columns[2:162]:
    data = last_train[x]
    l,r = st.t.interval(alpha=0.99, df=len(data)-1, loc=np.mean(data), scale=np.std(data, ddof=1) / np.sqrt(np.size(data)))
    le_ri.append(l)
    le_ri.append(r)
    last_train[x] = last_train[x].apply(per_fil)


In [ ]:
last_train.iloc[:,2:162]

,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,F2310,F2320,...,new3,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400
0,8.205209e+08,NaN,5.975429e+08,NaN,NaN,NaN,NaN,3.433044e+08,NaN,NaN,NaN,NaN,3.864020e+07,NaN,2.440075e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,NaN,1.050962e+09,8.576547e+08,3.689578e+07,NaN,NaN,2.979987e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175263,0.604512,2.124058e+06,0.072797,0.598534,1.388070e+06,NaN,NaN,NaN,NaN,NaN,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,NaN,NaN,NaN,5.974256e+06,1.031249e+08,8.772515e+07,9.180000e+05,NaN,NaN,918000.0,NaN,NaN,918000.0
1,8.205209e+08,NaN,5.975429e+08,NaN,1.089092e+09,NaN,NaN,3.433044e+08,6.303902e+07,NaN,2.065828e+08,NaN,3.864020e+07,NaN,2.440075e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299489e+08,1.868954e+08,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,NaN,1.050962e+09,8.576547e+08,3.689578e+07,NaN,NaN,2.979987e+07,NaN,NaN,...,779258.492949,NaN,49.828558,35.016473,NaN,NaN,NaN,NaN,NaN,NaN,0.175263,0.619048,2.124058e+06,0.072797,0.598534,2.130214e+06,NaN,NaN,NaN,NaN,NaN,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,7.408478e+06,NaN,8525000.0,5.974256e+06,1.031249e+08,8.772515e+07,1.168000e+06,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0
2,8.205209e+08,NaN,5.975429e+08,NaN,NaN,NaN,NaN,3.433044e+08,6.303902e+07,1.083694e+07,2.065828e+08,NaN,3.864020e+07,1.196981e+07,2.440075e+08,1.068996e+08,NaN,NaN,NaN,NaN,5.201243e+08,7.448575e+08,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,3.380253e+08,1.050962e+09,8.576547e+08,3.689578e+07,4.617717e+08,9.616489e+07,2.979987e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.175263,0.604512,NaN,0.072797,0.598534,NaN,NaN,NaN,NaN,NaN,NaN,6.258814,NaN,NaN,NaN,NaN,1.770512e+07,7.408478e+06,4.163157e+07,NaN,5.974256e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.205209e+08,NaN,5.975429e+08,NaN,NaN,7.023898e+07,4.662736e+08,3.433044e+08,6.303902e+07,NaN,2.065828e+08,1.728871e+08,3.864020e+07,NaN,2.440075e+08,1.068996e+08,NaN,NaN,NaN,1.329255e+08,NaN,NaN,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,5.105627e+07,7.418399e+08,7.424142e+08,3.380253e+08,1.050962e+09,8.576547e+08,4.023600e+07,4.617717e+08,NaN,2.979987e+07,NaN,3.198422e+07,...,779258.492949,64.069084,49.828558,31.244912,2.137783,0.638063,1.651511e+06,1.800571,0.634868,1.724681e+06,0.175263,0.620917,1.495016e+06,0.072797,0.613578,1.388070e+06,7.763312,NaN,17.240675,17.000000,162.362268,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,7.408478e+06,NaN,NaN,5.974256e+06,1.031249e+08,8.772515e+07,2.166701e+07,3.569230e+07,NaN,23302000.0,256000.0,NaN,19022000.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,779258.492949,NaN,54.968554,31.244912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.137783,0.618941,1

In [ ]:
last_train['random'] = [(10**6)*random.random() for i in range(8922)]

# VSE


In [ ]:
last_train.to_csv('last_df_mkb(14.11).csv')

In [ ]:
last_train.describe()

,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,F2310,...,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET,random
count,8922.000000,3.813000e+03,5.390000e+02,3.618000e+03,1.240000e+02,7.180000e+02,1.029000e+03,6.870000e+02,6.238000e+03,5.255000e+03,1.483000e+03,6.002000e+03,1.243000e+03,6.101000e+03,1.975000e+03,6.240000e+03,3.778000e+03,3.600000e+01,5.560000e+02,6.410000e+02,3.749000e+03,2.127000e+03,1.312000e+03,9.900000e+02,3.780000e+02,6.083000e+03,2.587000e+03,6.016000e+03,4.560000e+02,4.760000e+02,6.255000e+03,6.254000e+03,3.804000e+03,6.104000e+03,5.933000e+03,6.117000e+03,1.842000e+03,1.584000e+03,6.128000e+03,1.060000e+02,...,1315.000000,1492.000000,2253.000000,2204.000000,1.823000e+03,2791.000000,2737.000000,2.336000e+03,3608.000000,3599.000000,3.402000e+03,3753.000000,3749.000000,3.589000e+03,2253.000000,1255.000000,2075.000000,1974.000000,1972.000000,3611.000000,2218.000000,3533.000000,3464.000000,3461.000000,3.144000e+03,5.230000e+03,8.220000e+02,1.770000e+03,5.231000e+03,5.287000e+03,5.116000e+03,5.303000e+03,1.536000e+03,1.208000e+03,5.324000e+03,1.030000e+03,1.241000e+03,5.328000e+03,8922.000000,8922.000000
mean,8927.420745,8.847414e+08,5.463629e+07,6.453075e+08,5.728160e+07,1.126643e+09,7.397453e+07,4.935736e+08,3.645177e+08,6.784716e+07,1.163157e+07,2.194890e+08,1.820391e+08,4.123066e+07,1.335356e+07,2.853079e+08,1.314157e+08,-4.141325e+07,1.135532e+09,4.154268e+07,1.549897e+08,5.513886e+08,7.815852e+08,1.379094e+08,3.220192e+08,3.501633e+08,1.962315e+08,2.479143e+08,1.401846e+08,5.355429e+07,8.244061e+08,8.249748e+08,3.595142e+08,1.101178e+09,8.965140e+08,4.314890e+07,4.902647e+08,1.030932e+08,3.547102e+07,3.301396e+08,...,50.938641,31.841497,2.161581,0.629003,1.768838e+06,1.819817,0.626378,1.799074e+06,0.185815,0.612495,1.593485e+06,0.085635,0.605800,1.499713e+06,6.611448,57.017646,14.927138,15.258969,165.109434,6.642210,67.587796,18.943045,14.852028,72.314555,2.269221e+07,1.096001e+07,4.906157e+07,9.725946e+06,1.001458e+07,1.101307e+08,9.347399e+07,1.381874e+06,4.043476e+07,1.349182e+07,1.138645e+06,-2.089535e+05,-1.605725e+06,5.783352e+05,0.209786,502135.415625
std,4881.111898,2.734858e+08,9.247007e+06,1.950666e+08,5.132257e+06,1.593014e+08,9.843861e+06,1.211233e+08,6.257078e+07,1.411373e+07,2.165465e+06,3.876368e+07,2.831090e+07,9.529068e+06,5.065700e+06,1.646384e+08,1.110782e+08,2.739717e+06,1.160662e+08,1.077011e+07,6.618248e+07,1.366323e+08,1.551708e+08,2.371684e+07,3.128746e+07,6.241815e+07,3.464639e+07,4.067359e+07,1.062609e+07,1.015761e+07,2.931685e+08,2.931788e+08,7.643701e+07,1.604040e+08,1.186014e+08,2.259302e+07,9.198647e+07,1.837875e+07,2.160575e+07,3.486727e+07,...,2.115879,1.375639,0.140253,0.009506,2.528171e+05,0.119457,0.008849,1.615010e+05,0.049525,0.008164,2.220721e+05,0.041712,0.007473,2.551432e+05,0.394242,2.128547,0.701862,0.693967,8.801989,0.827375,2.580717,1.424045,1.095898,5.063497,1.663107e+07,6.985694e+06,2.471908e+07,1.530150e+07,7.917499e+06,1.914837e+07,1.501563e+07,8.110738e+06,1.376448e+07,1.989919e+06,1.232706e+07,1.710117e+06,3.312453e+06,1.119114e+07,0.402818,289313.209612
min,2.000000,8.205209e+08,5.183341e+07,5.975429e+08,5.509941e+07,1.089092e+09,7.023898e+07,4.662736e+08,3.433044e+08,6.303902e+07,1.083694e+07,2.065828e+08,1.728871e+08,3.864020e+07,1.196981e+07,2.440075e+08,1.068996e+08,-4.847270e+07,1.100175e+09,3.833354e+07,1.329255e+08,5.201243e+08,7.448575e+08,1.321079e+08,3.100162e+08,3.299489e+08,1.868954e+08,2.339708e+08,1.361542e+08,5.105627e+07,7.418399e+08,7.424142e+08,3.380253e+08,1.050962e+09,8.576547e+08,3.689578e+07,4.617717e+08,9.61

In [ ]:
def n05(x):
    if x>0.5:
        return 1
    if x==0.5:
        return random.choice([0, 1])
    return 0
labels_form = last_train.TARGET
last_train['TARGET'] = labels_form.apply(n05)

In [ ]:
N_THREADS = mp.cpu_count() # threads cnt for lgbm and linear models
N_FOLDS = 10 # folds cnt for AutoML - 10 is better
RANDOM_STATE = 42 # fixed random state for various reasons
TIMEOUT = 3600 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

In [ ]:
%%time

roles = {'target': TARGET_NAME,
         'drop': ['id_contract','id_client']
         }

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.4 µs


In [ ]:
%%time

task = Task('binary', loss='logloss', metric='auc')

CPU times: user 6.33 ms, sys: 1.03 ms, total: 7.36 ms
Wall time: 16.6 ms


In [ ]:
#### %%time 
automl = TabularAutoML(task = task, 
                       timeout = 1800,
                       cpu_limit = N_THREADS,
                       #general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(last_train, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Выходные данные были обрезаны до нескольких последних строк (5000).
[2021-11-14 14:25:09,228] (DEBUG): 4:	test: 0.7788979	best: 0.7788979 (4)	total: 71.8ms	remaining: 43s
[2021-11-14 14:25:09,246] (DEBUG): 5:	test: 0.7760041	best: 0.7788979 (4)	total: 86.7ms	remaining: 43.3s
[2021-11-14 14:25:09,265] (DEBUG): 6:	test: 0.7775515	best: 0.7788979 (4)	total: 104ms	remaining: 44.3s
[2021-11-14 14:25:09,280] (DEBUG): 7:	test: 0.7717298	best: 0.7788979 (4)	total: 116ms	remaining: 43.3s
[2021-11-14 14:25:09,296] (DEBUG): 8:	test: 0.7765313	best: 0.7788979 (4)	total: 129ms	remaining: 42.9s
[2021-11-14 14:25:09,314] (DEBUG): 9:	test: 0.7780862	best: 0.7788979 (4)	total: 145ms	remaining: 43.3s
[2021-11-14 14:25:09,330] (DEBUG): 10:	test: 0.7814351	best: 0.7814351 (10)	total: 158ms	remaining: 43s
[2021-11-14 14:25:09,348] (DEBUG): 11:	test: 0.7787462	best: 0.7814351 (10)	total: 173ms	remaining: 43.2s
[2021-11-14 14:25:09,367] (DEBUG): 12:	test: 0.7799939	best: 0.7814351 (10)	total: 189ms	remaining

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(last_train.TARGET,oof_pred.data)

0.8302164029823605

In [ ]:
all_featers = automl.get_feature_scores()

In [ ]:
main_features = automl.get_feature_scores()[:6].Feature.array

In [ ]:
main_features = list(main_features)

In [ ]:
main_features = main_features+['TARGET','id_client','id_contract']

In [ ]:
#### %%time 
automl = TabularAutoML(task = task, 
                       timeout = 1800,
                       cpu_limit = N_THREADS,
                       #general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(last_train[main_features], roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Выходные данные были обрезаны до нескольких последних строк (5000).
[2021-11-14 14:47:00,552] (DEBUG): 131:	test: 0.7987098	best: 0.7995322 (129)	total: 1.52s	remaining: 55.9s
[2021-11-14 14:47:00,561] (DEBUG): 132:	test: 0.7985665	best: 0.7995322 (129)	total: 1.52s	remaining: 55.8s
[2021-11-14 14:47:00,574] (DEBUG): 133:	test: 0.7987702	best: 0.7995322 (129)	total: 1.53s	remaining: 55.7s
[2021-11-14 14:47:00,592] (DEBUG): 134:	test: 0.7985891	best: 0.7995322 (129)	total: 1.55s	remaining: 55.8s
[2021-11-14 14:47:00,610] (DEBUG): 135:	test: 0.7986419	best: 0.7995322 (129)	total: 1.56s	remaining: 55.8s
[2021-11-14 14:47:00,623] (DEBUG): 136:	test: 0.7992153	best: 0.7995322 (129)	total: 1.57s	remaining: 55.7s
[2021-11-14 14:47:00,636] (DEBUG): 137:	test: 0.7995096	best: 0.7995322 (129)	total: 1.58s	remaining: 55.7s
[2021-11-14 14:47:00,651] (DEBUG): 138:	test: 0.7995171	best: 0.7995322 (129)	total: 1.59s	remaining: 55.7s
[2021-11-14 14:47:00,671] (DEBUG): 139:	test: 0.7998566	best: 0.7998

In [ ]:
roc_auc_score(last_train.TARGET,oof_pred.data)

0.8263194444444444

In [ ]:
automl.get_feature_scores()

,Feature,Importance
0,OKVED_CODE_fixed,32581.085720
1,WORKERSRANGE,23872.405050
2,F2410,12375.988617
3,OKTMO_CODE_fixed,11993.295093
4,OKATO_REGIONCODE_y,8287.667231
5,new17,7997.188044


In [ ]:
test_df = l_d[17891:]

In [ ]:
test_df

,id_contract,id_client,IP_flag,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,...,new3,new4,new5,new6,new7,new8,new9,new10,new11,new12,new13,new14,new15,new16,new17,new18,new19,new20,new21,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400
17891,17892,3620,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17892,17893,4101,0,1906000.0,NaN,1906000.0,NaN,NaN,NaN,NaN,14838000.0,4392000.0,NaN,727000.0,NaN,9719000.0,NaN,15222000.0,150000.0,NaN,NaN,NaN,15072000.0,NaN,NaN,NaN,NaN,1522000.0,NaN,1522000.0,NaN,NaN,16744000.0,16744000.0,2998000.0,310151000.0,307153000.0,2998000.0,NaN,NaN,...,NaN,NaN,NaN,3.5,0.000000,0.425414,380500.553191,0.011844,0.498308,317635.836773,0.001661,0.711794,1.801756e+05,0.006738,0.726465,1.672409e+05,6.530387,inf,7.649351,11.340426,9.466808,8.504983,34.5,8.680280,9.256845,8.592306,-816000.0,-1312000.0,NaN,NaN,-13679000.0,-16752000.0,-15658000.0,-1094000.0,NaN,NaN,-1094000.0,NaN,NaN,-876000.0
17893,17894,9589,0,147000.0,NaN,147000.0,NaN,NaN,NaN,NaN,95529000.0,29812000.0,7000.0,56538000.0,504000.0,8430000.0,238000.0,4145000.0,2538000.0,NaN,NaN,NaN,1607000.0,NaN,NaN,NaN,NaN,91531000.0,NaN,91531000.0,NaN,NaN,95676000.0,95676000.0,48975000.0,506023000.0,457048000.0,1753000.0,47222000.0,NaN,...,124509.285714,inf,7.0,10.3,0.064516,0.451613,334651.818182,0.135593,0.412429,731514.666667,0.277778,0.194444,3.393157e+05,0.065445,0.473822,2.859378e+05,5.709677,12.0,5.214286,5.727273,12.519233,21.222222,5.0,51.714286,49.142857,41.412171,-952000.0,-66171000.0,NaN,NaN,-75373000.0,355987000.0,339989000.0,466000.0,15532000.0,NaN,749000.0,NaN,NaN,683000.0
17894,17895,11546,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1906000.0,304000.0,NaN,682000.0,NaN,920000.0,NaN,1038000.0,NaN,NaN,NaN,NaN,NaN,95000.0,95000.0,NaN,NaN,774000.0,NaN,774000.0,NaN,NaN,1907000.0,1907000.0,NaN,9793000.0,8731000.0,1062000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.500000,NaN,0.000000,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561000.0,0.0,NaN,468000.0,8315000.0,7378000.0,937000.0,NaN,NaN,914000.0,NaN,NaN,858000.0
17895,17896,12558,0,26000.0,NaN,NaN,NaN,26000.0,NaN,NaN,2735000.0,1324000.0,NaN,1410000.0,NaN,1000.0,NaN,1220000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1541000.0,179000.0,1362000.0,NaN,NaN,2761000.0,2761000.0,NaN,6407000.0,6301000.0,106000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,623000.0,NaN,59000.0,11000.0,355000.0,802000.0,-447000.0,NaN,NaN,-445000.0,NaN,NaN,-356000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25216,25217,8128,0,1395000.0,NaN,1395000.0,NaN,NaN,NaN,NaN,515000.0,163000.0,NaN,318000.0,NaN,34000.0,NaN,44000.0,NaN,NaN,NaN,NaN,NaN,726000.0,NaN,NaN,726000.0,1140000.0,300000.0,840000.0,NaN,NaN,1910000.0,1910000.0,NaN,1518000.0,1424000.0,94000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,308000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25217,25218,1132,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
test_df1 = test_df[konh.drop(['TARGET'],axis=1).columns]

In [ ]:
obj_df = test_df.loc[:,['id_client','IP_flag','EGRPOINCLUDED','OKFS_GROUP','OKOPF_GROUP','OKOGU_GROUP','OKATO_REGIONCODE','OKATO_FED','OKTMO_CODE_fixed','OKTMO_FED','WORKERSRANGE','OKVED_CODE_fixed','FLAG_DISQUALIFICATION','SEX_NAME','CITIZENSHIP_NAME','TELEPHONECOUNT_fixed','MANAGERCOUNTINCOUNTRY_fixed','MANAGERCOUNTINREGION_fixed','MANAGERINNCOUNT_fixed']]

In [ ]:
obj_df = obj_df.astype('category')

In [ ]:
obj_df = obj_df.drop(['id_client'],axis=1)

In [ ]:
obj_df['id_contract'] = test_df1['id_contract']

In [ ]:
last_test = pd.merge(test_df1, obj_df, on='id_contract', how='outer')

In [ ]:
obj_df['id_contract'] = last_test['id_contract']

In [ ]:
set(last_test.columns)-set(last_train.columns)

{'EGRPOINCLUDED_x',
 'FLAG_DISQUALIFICATION_x',
 'IP_flag_x',
 'OKATO_FED_x',
 'OKATO_REGIONCODE_x',
 'OKTMO_FED_x'}

In [ ]:
last_test = last_test.drop(['EGRPOINCLUDED_x',
 'FLAG_DISQUALIFICATION_x',
 'IP_flag_x',
 'OKATO_FED_x',
 'OKATO_REGIONCODE_x',
#3333 'OKTMO_CODE',
 'OKTMO_FED_x',
# 'id_client_x',
# 'id_client_y'
],axis=1)

In [ ]:
last_test['id_client'] = test_df1['id_client'].array

In [ ]:
test_df1['id_client'].array

<PandasArray>
[ 3620,  4101,  9589, 11546, 12558,  7933,  8980,  2582,  7495,  8429,
 ...
  5661, 11496,  8684, 11021, 10079,  8128,  1132,  4932,   537,  9916]
Length: 7330, dtype: int64

In [ ]:
last_test[main_features_test]

,OKVED_CODE_fixed,WORKERSRANGE,OKTMO_CODE_fixed,F2410,new17,OKATO_REGIONCODE_y,id_client,id_contract
0,NaN,NaN,NaN,NaN,NaN,NaN,3620,17892
1,46.3,16 .. 50,RARE_VALUE,600000.0,0.726465,24.0,4101,17893
2,33.12,16 .. 50,75701390000.0,371000.0,0.473822,75.0,9589,17894
3,RARE_VALUE,0 .. 5,RARE_VALUE,94000.0,NaN,40.0,11546,17895
4,43.33,0 .. 5,67310000000.0,18000.0,NaN,67.0,12558,17896
...,...,...,...,...,...,...,...,...
7325,73.11,0 .. 5,63701000001.0,15000.0,0.000000,63.0,8128,25217
7326,35.14,101 .. 150,92608101001.0,NaN,NaN,92.0,1132,25218
7327,43.21,0 .. 5,37701000001.0,98000.0,0.607143,37.0,4932,25219
7328,46.17,0 .. 5,85701000001.0,21000.0,1.000000,85.0,537,25220


In [ ]:
main_features_test = ['OKVED_CODE_fixed',
 'WORKERSRANGE',
 'OKTMO_CODE_fixed',
 'F2410',
 'new17',
 'OKATO_REGIONCODE_y',
 
 'id_client',
 'id_contract']

In [ ]:
test_pred = automl.predict(last_test[main_features_test])

In [ ]:
test_hren = last_test[main_features_test]

In [ ]:
test_hren['TARGET'] = test_pred.data

In [ ]:
test_hren['TARGET'] = test_hren['TARGET'].apply(n05)

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new22,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed,TARGET
0,3620,17892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,4101,17893,1906000.0,NaN,1906000.0,NaN,NaN,NaN,NaN,14838000.0,4392000.0,NaN,727000.0,NaN,9719000.0,NaN,15222000.0,150000.0,NaN,NaN,NaN,15072000.0,NaN,NaN,NaN,NaN,1522000.0,NaN,1522000.0,NaN,NaN,16744000.0,16744000.0,2998000.0,310151000.0,307153000.0,2998000.0,NaN,NaN,2998000.0,...,11.340426,9.466808,8.504983,34.5,8.680280,9.256845,8.592306,-816000.0,-1312000.0,NaN,NaN,-13679000.0,-16752000.0,-15658000.0,-1094000.0,NaN,NaN,-1094000.0,NaN,NaN,-876000.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,24.0,30.0,RARE_VALUE,30.0,16 .. 50,46.3,NaN,NaN,NaN,5.0,2.0,2.0,2.0,0
2,9589,17894,147000.0,NaN,147000.0,NaN,NaN,NaN,NaN,95529000.0,29812000.0,7000.0,56538000.0,504000.0,8430000.0,238000.0,4145000.0,2538000.0,NaN,NaN,NaN,1607000.0,NaN,NaN,NaN,NaN,91531000.0,NaN,91531000.0,NaN,NaN,95676000.0,95676000.0,48975000.0,506023000.0,457048000.0,1753000.0,47222000.0,NaN,1771000.0,...,5.727273,12.519233,21.222222,5.0,51.714286,49.142857,41.412171,-952000.0,-66171000.0,NaN,NaN,-75373000.0,355987000.0,339989000.0,466000.0,15532000.0,NaN,749000.0,NaN,NaN,683000.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,75.0,34.0,75701390000.0,34.0,16 .. 50,33.12,NaN,NaN,NaN,3.0,1.0,1.0,1.0,0
3,11546,17895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1906000.0,304000.0,NaN,682000.0,NaN,920000.0,NaN,1038000.0,NaN,NaN,NaN,NaN,NaN,95000.0,95000.0,NaN,NaN,774000.0,NaN,774000.0,NaN,NaN,1907000.0,1907000.0,NaN,9793000.0,8731000.0,1062000.0,NaN,NaN,1027000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,561000.0,0.0,NaN,468000.0,8315000.0,7378000.0,937000.0,NaN,NaN,914000.0,NaN,NaN,858000.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,40.0,31.0,RARE_VALUE,31.0,0 .. 5,RARE_VALUE,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0
4,12558,17896,26000.0,NaN,NaN,NaN,26000.0,NaN,NaN,2735000.0,1324000.0,NaN,1410000.0,NaN,1000.0,NaN,1220000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1541000.0,179000.0,1362000.0,NaN,NaN,2761000.0,2761000.0,NaN,6407000.0,6301000.0,106000.0,NaN,NaN,90000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,623000.0,NaN,59000.0,11000.0,355000.0,802000.0,-447000.0,NaN,NaN,-445000.0,NaN,NaN,-356000.0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,67.0,40.0,67310000000.0,40.0,0 .. 5,43.33,NaN,NaN,NaN,1.0,3.0,2.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7325,8128,25217,1395000.0,NaN,1395000.0,NaN,NaN,NaN,NaN,515000.0,163000.0,NaN,318000.0,NaN,34000.0,NaN,44000.0,NaN,NaN,NaN,NaN,NaN,726000.0,NaN,NaN,726

In [ ]:
train_hren = last_train[main_features]

In [ ]:
df_hren = pd.concat([train_hren,test_hren])

In [ ]:
#### %%time 
automl = TabularAutoML(task = task, 
                       timeout = 1800,
                       cpu_limit = N_THREADS,
                       #general_params = {'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                      )
oof_pred = automl.fit_predict(df_hren, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Выходные данные были обрезаны до нескольких последних строк (5000).
[2021-11-14 15:52:01,684] (DEBUG): 1019:	test: 0.8595379	best: 0.8613349 (929)	total: 11.5s	remaining: 22.4s
[2021-11-14 15:52:01,703] (DEBUG): 1020:	test: 0.8595673	best: 0.8613349 (929)	total: 11.5s	remaining: 22.4s
[2021-11-14 15:52:01,723] (DEBUG): 1021:	test: 0.8596932	best: 0.8613349 (929)	total: 11.5s	remaining: 22.4s
[2021-11-14 15:52:01,740] (DEBUG): 1022:	test: 0.8597575	best: 0.8613349 (929)	total: 11.6s	remaining: 22.3s
[2021-11-14 15:52:01,760] (DEBUG): 1023:	test: 0.8599342	best: 0.8613349 (929)	total: 11.6s	remaining: 22.3s
[2021-11-14 15:52:01,777] (DEBUG): 1024:	test: 0.8599342	best: 0.8613349 (929)	total: 11.6s	remaining: 22.3s
[2021-11-14 15:52:01,795] (DEBUG): 1025:	test: 0.8599824	best: 0.8613349 (929)	total: 11.6s	remaining: 22.3s
[2021-11-14 15:52:01,810] (DEBUG): 1026:	test: 0.8600039	best: 0.8613349 (929)	total: 11.6s	remaining: 22.3s
[2021-11-14 15:52:01,831] (DEBUG): 1027:	test: 0.8599798	bes

In [ ]:
df_hren

,OKVED_CODE_fixed,WORKERSRANGE,OKTMO_CODE_fixed,F2410,new17,OKATO_REGIONCODE_y,TARGET,id_client,id_contract
0,43.11,6 .. 10,RARE_VALUE,NaN,0.598534,79.0,1,1,5166.0
1,31.09,6 .. 10,RARE_VALUE,425000.0,0.598534,79.0,0,2,2389.0
2,10.11,0 .. 5,RARE_VALUE,NaN,0.598534,79.0,1,3,17329.0
3,10.51,251 .. 500,79701000001.0,5915000.0,0.613578,79.0,0,5,15086.0
4,47.52.7,NaN,79701000001.0,NaN,NaN,79.0,0,7,933.0
...,...,...,...,...,...,...,...,...,...
7325,73.11,0 .. 5,63701000001.0,15000.0,0.000000,63.0,0,8128,25217.0
7326,35.14,101 .. 150,92608101001.0,NaN,NaN,92.0,0,1132,25218.0
7327,43.21,0 .. 5,37701000001.0,98000.0,0.607143,37.0,0,4932,25219.0
7328,46.17,0 .. 5,85701000001.0,21000.0,1.000000,85.0,0,537,25220.0


In [ ]:
lab_df = pd.DataFrame()

In [ ]:
lab_df['id_client'] = konh['id_client']

In [ ]:
lab_df['TARGET'] = labels_form

In [ ]:
id = list(lab_df['id_client'].array)
tar = list(lab_df['TARGET'].array)

In [ ]:
trans = dict(zip(id,tar))

In [ ]:
test_df_otpravka = pd.DataFrame()

In [ ]:
test_df_otpravka['id_client'] = test_df['id_client']
test_df_otpravka['id_contract'] = test_df['id_contract']

In [ ]:
tar_pred = oof_pred.data[8922:]

In [ ]:
len(tar_pred)

7330

In [ ]:
id_test = list(test_df_otpravka['id_client'].array)

In [ ]:
new_tar2 = []

In [ ]:
for i in range(len(id_test)):
    if id_test[i] in id:
        new_tar2.append(0.3*trans[id_test[i]]+0.7*tar_pred[i])
    else:
        new_tar2.append(tar_pred[i])

In [ ]:
tar_pred = list(tar_pred.squeeze())

In [ ]:
test_df_otpravka['TARGET']=new_tar

In [ ]:
test_df_otpravka

,id_client,id_contract,TARGET
17891,3620,17892,0.004981
17892,4101,17893,0.000571
17893,9589,17894,0.013926
17894,11546,17895,0.092958
17895,12558,17896,0.972740
...,...,...,...
25216,8128,25217,0.623815
25217,1132,25218,0.001420
25218,4932,25219,0.091747
25219,537,25220,0.017349


In [ ]:
len(tar_pred)

7330

In [ ]:
test_df_otpravka.drop(['id_client'],axis=1).to_csv('trying_6_features2.csv', index=False,sep=';')

In [ ]:
test_df_otpravka.drop(['id_client'],axis=1)

,id_contract,TARGET
17891,17892,0.009963
17892,17893,0.001142
17893,17894,0.013926
17894,17895,0.092958
17895,17896,0.945480
...,...,...
25216,25217,0.623815
25217,25218,0.002840
25218,25219,0.091747
25219,25220,0.017349


In [ ]:
last_train

,id_client,id_contract,F1100,F1110,F1150,F1160,F1170,F1180,F1190,F1200,F1210,F1220,F1230,F1240,F1250,F1260,F1300,F1310,F1320,F1350,F1360,F1370,F1400,F1410,F1420,F1450,F1500,F1510,F1520,F1530,F1550,F1600,F1700,F2100,F2110,F2120,F2200,F2210,F2220,F2300,...,new23,new24,new25,new26,new27,new28,new_F1150,new_F1230,new_F1410,new_F1510,new_F1520,new_F2110,new_F2120,new_F2200,new_F2210,new_F2220,new_F2300,new_F2320,new_F2330,new_F2400,TARGET,IP_flag_y,EGRPOINCLUDED_y,OKFS_GROUP,OKOPF_GROUP,OKOGU_GROUP,OKATO_REGIONCODE_y,OKATO_FED_y,OKTMO_CODE_fixed,OKTMO_FED_y,WORKERSRANGE,OKVED_CODE_fixed,FLAG_DISQUALIFICATION_y,SEX_NAME,CITIZENSHIP_NAME,TELEPHONECOUNT_fixed,MANAGERCOUNTINCOUNTRY_fixed,MANAGERCOUNTINREGION_fixed,MANAGERINNCOUNT_fixed,random
0,1,5166.0,8.205209e+08,NaN,5.975429e+08,NaN,NaN,NaN,NaN,3.433044e+08,NaN,NaN,NaN,NaN,3.864020e+07,NaN,2.440075e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,NaN,1.050962e+09,8.576547e+08,3.689578e+07,NaN,NaN,2.979987e+07,...,NaN,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,NaN,NaN,NaN,5.974256e+06,1.031249e+08,8.772515e+07,9.180000e+05,NaN,NaN,918000.0,NaN,NaN,918000.0,1,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,43.11,NaN,NaN,NaN,1.0,1.0,1.0,1.0,183264.497772
1,2,2389.0,8.205209e+08,NaN,5.975429e+08,NaN,1.089092e+09,NaN,NaN,3.433044e+08,6.303902e+07,NaN,2.065828e+08,NaN,3.864020e+07,NaN,2.440075e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.299489e+08,1.868954e+08,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,NaN,1.050962e+09,8.576547e+08,3.689578e+07,NaN,NaN,2.979987e+07,...,NaN,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,7.408478e+06,NaN,8525000.0,5.974256e+06,1.031249e+08,8.772515e+07,1.168000e+06,NaN,NaN,1368000.0,NaN,-200000.0,1095000.0,0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,6 .. 10,31.09,NaN,NaN,NaN,2.0,1.0,1.0,1.0,869898.358417
2,3,17329.0,8.205209e+08,NaN,5.975429e+08,NaN,NaN,NaN,NaN,3.433044e+08,6.303902e+07,1.083694e+07,2.065828e+08,NaN,3.864020e+07,1.196981e+07,2.440075e+08,1.068996e+08,NaN,NaN,NaN,NaN,5.201243e+08,7.448575e+08,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,NaN,7.418399e+08,7.424142e+08,3.380253e+08,1.050962e+09,8.576547e+08,3.689578e+07,4.617717e+08,9.616489e+07,2.979987e+07,...,NaN,6.258814,NaN,NaN,NaN,NaN,1.770512e+07,7.408478e+06,4.163157e+07,NaN,5.974256e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,RARE_VALUE,40.0,0 .. 5,10.11,NaN,NaN,NaN,2.0,1.0,1.0,1.0,558035.700623
3,5,15086.0,8.205209e+08,NaN,5.975429e+08,NaN,NaN,7.023898e+07,4.662736e+08,3.433044e+08,6.303902e+07,NaN,2.065828e+08,1.728871e+08,3.864020e+07,NaN,2.440075e+08,1.068996e+08,NaN,NaN,NaN,1.329255e+08,NaN,NaN,NaN,NaN,3.299489e+08,NaN,2.339708e+08,NaN,5.105627e+07,7.418399e+08,7.424142e+08,3.380253e+08,1.050962e+09,8.576547e+08,4.023600e+07,4.617717e+08,NaN,2.979987e+07,...,162.362268,6.258814,65.996851,18.635958,14.574904,71.475694,1.770512e+07,7.408478e+06,NaN,NaN,5.974256e+06,1.031249e+08,8.772515e+07,2.166701e+07,3.569230e+07,NaN,23302000.0,256000.0,NaN,19022000.0,0,0,1.0,Частная собственность,Коммерческая корпоративная организация,Группировки хозяйствующих субъектов и обществе...,79.0,40.0,79701000001.0,40.0,251 .. 500,10.51,NaN,NaN,NaN,2.0,1.0,1.0,1.0,116989.998743
4,7,933.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,79.0,40.0,79701000001.0,40.0,NaN,47.52.7,NaN,мужской,Российская Федерация,NaN,NaN,NaN,NaN,465947.531773
...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
feat= all_featers[:30]

In [ ]:
feat = feat.Feature.drop([6])

In [ ]:
feat = list(feat.array)

In [ ]:
last_test1 = last_test[feat]
last_train1 = last_train[feat]

In [ ]:
last_train1.describe()

,F2410,new17,new_TAXREG_REGDATE_year,new16,new_F2200,new1,new_F2400,new_F2300,new_DATEFIRSTREG_year,new_F1150,new15,SIGNEDNUMBER_95_EVER,AGE,new_F1520,new14,F2340,new25,F2200_LAG1,F1210,new_F2120,new8
count,4.596000e+03,3749.000000,8492.000000,3753.000000,5.303000e+03,1.044000e+03,5.328000e+03,5.324000e+03,8494.000000,3.144000e+03,3.402000e+03,3753.000000,872.000000,5.231000e+03,3599.000000,3.993000e+03,2218.000000,5.329000e+03,5.255000e+03,5.116000e+03,2204.000000
mean,2.127388e+06,0.605800,2010.533822,0.085635,1.381874e+06,1.111173e+06,5.783352e+05,1.138645e+06,2009.302114,2.269221e+07,1.593485e+06,165.890408,44.269166,1.001458e+07,0.612495,1.115881e+08,67.587796,4.902157e+07,6.784716e+07,9.347399e+07,0.629003
std,4.056150e+06,0.007473,0.145826,0.041712,8.110738e+06,1.163049e+05,1.119114e+07,1.232706e+07,0.200137,1.663107e+07,2.220721e+05,13.994556,0.293329,7.917499e+06,0.008164,2.105869e+07,2.580717,2.322738e+07,1.411373e+07,1.501563e+07,0.009506
min,-3.420028e+06,0.598534,2010.370823,0.072797,-1.125127e+07,1.065613e+06,-3.124209e+07,-3.001151e+07,2009.064682,1.770512e+07,1.495016e+06,158.826058,44.004237,5.974256e+06,0.604512,1.048553e+08,65.996851,4.256947e+07,6.303902e+07,8.772515e+07,0.618941
25%,1.013750e+05,0.598534,2010.370823,0.072797,-1.602500e+06,1.065613e+06,-1.084250e+06,-1.075500e+06,2009.064682,1.770512e+07,1.495016e+06,158.826058,44.004237,5.974256e+06,0.604512,1.048553e+08,65.996851,4.256947e+07,6.303902e+07,8.772515e+07,0.618941
50%,4.250000e+05,0.598534,2010.664269,0.072797,1.990000e+05,1.065613e+06,1.225000e+05,1.865000e+05,2009.470793,1.770512e+07,1.495016e+06,158.826058,44.004237,5.974256e+06,0.604512,1.048553e+08,65.996851,4.256947e+07,6.303902e+07,8.772515e+07,0.638063
75%,1.642000e+06,0.613578,2010.664269,0.072797,3.031000e+06,1.065613e+06,2.344250e+06,2.683250e+06,2009.470793,1.770512e+07,1.495016e+06,158.826058,44.593568,7.297500e+06,0.620917,1.048553e+08,71.772163,4.256947e+07,6.303902e+07,8.772515e+07,0.638063
max,1.425500e+07,0.613578,2010.664269,0.276333,2.166701e+07,1.422255e+06,3.059969e+07,3.581423e+07,2009.470793,8.787821e+07,2.124058e+06,194.617084,44.593568,2.783729e+07,0.620917,1.819016e+08,71.772163,1.449796e+08,1.125815e+08,1.348571e+08,0.638063


['OKVED_CODE_fixed',
 'WORKERSRANGE',
 'OKTMO_CODE_fixed',
 'F2410',
 'new17',
 'OKATO_REGIONCODE_y',
 'new_TAXREG_REGDATE_year',
 'new16',
 'OKFS_GROUP',
 'new_F2200',
 'TELEPHONECOUNT_fixed',
 'OKATO_FED_y',
 'new1',
 'new_F2400',
 'new_F2300',
 'new_DATEFIRSTREG_year',
 'new_F1150',
 'new15',
 'SIGNEDNUMBER_95_EVER',
 'AGE',
 'new_F1520',
 'new14',
 'MANAGERCOUNTINCOUNTRY_fixed',
 'F2340',
 'new25',
 'F2200_LAG1',
 'F1210',
 'new_F2120',
 'new8']